# FIREX Campaign Spatial Intersect Exercise

In this exercise we are going to be looking at data from the FIREX campaign and comparing it with wildfire data shapes from the USGS. The `geopandas` lesson page will be a helpful reference. https://rwegener2.github.io/sarp_lessons/lessons/tabular_data/3_geopandas.html

In [3]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [4]:
firex_all = pd.read_csv('../data/firexaq-mrg60-dc8_merge_20190722_R1_thru20190905.csv', skiprows=676)

_Note: This file was originally a `.ict` file but I used the File Explorer to re-name it to a `.csv` file. Pandas can open `.ict` files just fine, but I find that Excel has a hard time with them. If you rename the same file from `.ict` to `.csv`, however, Excel will open it up._

In [68]:
firex_all.describe()

,Fractional_Day,Time_Start,Time_Stop,Day_Of_Year_YANG,Latitude_YANG,Longitude_YANG,MSL_GPS_Altitude_YANG,HAE_GPS_Altitude_YANG,Pressure_Altitude_YANG,Radar_Altitude_YANG,...,jBrONO2_BrO_NO2_CAFS_HALL,jBrONO2_Br_NO3_CAFS_HALL,jBrCl_Br_Cl_CAFS_HALL,jCHBr3_NoProductsSpecified_CAFS_HALL,smoke_age_HOLMES,smoke_age_corr_HOLMES,smoke_age_rise_HOLMES,smoke_rise_HOLMES,fire_distance_HOLMES,smoke_agemethod_HOLMES
count,9081.000000,9081.000000,9081.000000,9081.000000,9081.000000,9081.000000,9081.000000,9081.000000,9081.000000,9081.000000,...,9081.000000,9081.000000,9081.000000,9.081000e+03,9081.000000,9081.000000,9081.000000,9081.000000,9081.000000,9081.000000
mean,226.760197,81625.678890,81685.678890,225.815108,-1722.146676,-1869.220822,3156.048955,-2292.452929,15359.291967,12676.498769,...,-330.359580,-330.358731,-330.350287,-3.303598e+02,-822610.049450,-822536.417093,-826164.556287,-825761.166757,-800607.897810,-826244.062768
std,13.324944,11555.139635,11555.139635,13.390380,41942.180600,41936.002445,42262.242721,84775.483466,9758.671210,20621.489347,...,18173.790620,18173.790636,18173.790789,1.817379e+04,386958.865866,387131.448999,379073.762924,379953.825504,444493.726463,378901.670616
min,203.762500,52470.000000,52530.000000,203.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,508.533300,-999999.000000,...,-999999.000000,-999999.000000,-999999.000000,-9.999990e+05,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000
25%,215.930556,73230.000000,73290.000000,215.000000,34.776165,-116.700670,2541.030000,2382.890000,7842.780000,5530.440000,...,0.000104,0.000588,0.006327,1.928000e-07,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000
50%,226.048611,81930.000000,81990.000000,225.000000,38.273844,-112.260533,4742.480000,4663.480000,14643.200000,10821.500000,...,0.000207,0.001170,0.010991,8.174000e-07,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000
75%,238.902778,90030.000000,90090.000000,238.000000,45.356357,-95.753198,7715.070000,7685.270000,23975.300000,20293.000000,...,0.000256,0.001452,0.012980,1.246000e-06,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000,-999999.000000
max,248.965972,107670.000000,107730.000000,248.000000,48.971215,-83.123175,12502.500000,12475.500000,38978.600000,39361.400000,...,0.000634,0.003593,0.032215,3.133000e-06,112476.600000,206980.500000,796.850000,9099.080000,932455.400000,7.000000


In [69]:
firex_all = firex_all.replace({-999999: np.nan, -888888: np.nan, -66666: np.nan})

## Processing the FIREX Dataset

1. Convert the pandas dataframe into a geodataframe. Be sure to specify the CRS as `epsg:4326`.

2. How many rows and columns are in this dataframe?

3. We have a pretty big dataframe. Let's filter it down to just the following columns:
```
['Fractional_Day', ' Time_Start', ' Time_Stop', ' Day_Of_Year_YANG',
       ' Latitude_YANG', ' Longitude_YANG', ' Pressure_Altitude_YANG', ' Potential_Temp_YANG', 
       ' Sat_Vapor_Press_H2O_YANG', ' Smoke_flag_SCHWARZ', ' BC_mass_90_550_nm_SCHWARZ', 'geometry']
```

4. To keep filtering, let's filter it down to just the Fractional Day 203.

## Processing the Wildfires Shapefile

1. The `data` folder has a file at `wildfires_2019_usgs/wildfires_2019_usgs.shp` which contains all of the burn areas of fires in 2019. Open that file up using `gpd.read_file()` and give the filepath to that `.shp` file as an argument. Assign output to a variable called `wildfires`.

2. Look at the new dataframe. How many rows and columns does it have? What is the type of the geometry (Point, Polygon, Multipolygon, Line, etc.)?

3. What is the CRS of this dataframe?

4. If you run `wildfires.plot()` you'll notice that the points are very difficult or impossible to see. To make the shapes easier to see let's make them larger, or buffer them by 100,000m. Use the `.buffer(100000)` method on the dataframe and then plot it to see the shape of the data more clearly.

This image visually shows what a buffer operation does to points, lines, and polygons.

![Buffers](https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/GUID-267CF0D1-DB92-456F-A8FE-F819981F5467-web.png)

5. From that shape we can see that the wildfire data is spread out between the contiguous US (CONUS) Hawai'i and Alaska. To make our file more manegable let's cut it down to only the fires in CONUS.

In order to do that we are going to create a new shape that covers the area we want to keep and intersect the two areas. We are going to create a shape that is a box using `shapely`.

In [21]:
from shapely.geometry import box

In [ ]:
conus = box(-3000000, -2000000, 2000000, 2000000)  # In units of the dataset

Now that we have a shape `conus` defined, use the `.intersects()` method on the `wildfires` dataframe with `conus` as an argument. This will return a True/False boolean array indicating if each row in the dataframe intersects the `conus` shape.

Use the True/False dataframe from the previous step to filter the `wildfires` dataframe.  Assign this dataframe to a new variable called `wildfires_conus`.

6. Plot the new `wildfires_conus` dataframe again, again buffering by 10000m, to confirm that you have spatially filtered the dataframe.

Nice work! You have used two spatial data operations -- buffer and intersects -- in Python AND used those to spatially filter a dataframe!